In [ ]:
import build123d as bd
import gggears as gg
import pathlib

class Gripper:
    def __init__(self):
        self.gripper_arm_joint_radius = 1
        self.gripper_height = 30
        self.gripper_length = 30
        self.top_depth = 50
        self.top_width = 50
        self.top_thickness = 4
        self.wedge_height = self.top_thickness * 0.4
        self.motor_height = 15
        self.motor_width = 21
        self.motor_length = 31
        self.motor_shaft_radius = 1.5/2
        self.arm_gear_height = 5
        self.gear_module_factor = 0.06
        self.worm_height = 5
        
        self.gear_module = self.worm_height * self.gear_module_factor

        self.worm_gear = gg.HelicalGear(
                number_of_teeth=5, helix_angle=gg.PI / 2 * 0.75, height=self.worm_height, module=self.gear_module, backlash=0.1
        )

        self.arm_gear = gg.HelicalGear(
            number_of_teeth=45, helix_angle=gg.PI / 2 * 0.25, height=self.worm_height, module=self.gear_module, backlash=0.1
        )

        self.worm_gear_arm_length = self.arm_gear.max_outside_radius

    def make_gripper_base(self):
        line = bd.ThreePointArc((0,0),(-self.gripper_length/2,self.gripper_length/2),(0,self.gripper_length))
        face = bd.SlotArc(line, 3)
        arm = bd.extrude(face, self.gripper_height)

        for i in range(3):
            arm -= bd.Pos(0,40, i * (self.gripper_height/3) + 3 ) * bd.Box(60,60,6)

        arm += bd.Cylinder(1, self.top_thickness)
        arm += bd.Cylinder(1, self.top_thickness).translate((0, 0, self.gripper_height))
        return arm

    def make_gripper_left(self):
        arm = self.make_gripper_base()

        gear_part = self.arm_gear.build_part()
        arm += gear_part.translate((0,0, self.motor_height/2))
        
        return arm

    def make_gripper_right(self):
        arm = bd.Part() + self.make_gripper_base()
        arm = arm.rotate(bd.Axis.Y, 180).translate((0,0,self.gripper_height))


        gear_part = self.arm_gear.build_part()
        arm += gear_part.translate((0,0, self.motor_height/2))

        return arm
    
    def worm_drive(self):
        drive = bd.Part() + self.worm_gear.build_part()

        arm_shift = -self.worm_gear_arm_length/2 
        drive += bd.Cylinder(self.worm_gear.dedendum_radius, self.worm_gear_arm_length)\
                        .translate((0,0, arm_shift))
        
        drive -= bd.Cylinder(self.motor_shaft_radius, self.worm_gear_arm_length)\
                        .translate((0,0, arm_shift))

        return drive

    def plate_wedge(self, length = None, tolerance_term = 0):
        if length is None:
            length = self.top_depth

        z = self.wedge_height / 2

        line = bd.Polyline([(
            -z-tolerance_term,-z),
            (z+tolerance_term,-z),
            (2*z+tolerance_term,z), 
            (-2*z-tolerance_term,z) ,
            (-z-tolerance_term,-z)
        ])
        
        face = bd.make_face(bd.Plane.XZ * line)
        wedge = bd.extrude(face, length)

        
        return wedge.translate((0,length/2,0))

    def holes_y_pos(self):
        return self.top_depth/2 - self.top_thickness - self.gripper_arm_joint_radius

    def top_plate(self):
        res = bd.Part() + bd.Box(self.top_width, self.top_depth, 4)
        holes_y_pos = self.holes_y_pos()
        hole_translate = self.arm_gear.pitch_radius + self.worm_gear.pitch_radius
        r = self.gripper_arm_joint_radius + 1 #tolerance 1mm
        res -= bd.Cylinder(r, self.top_thickness).translate((hole_translate, holes_y_pos, 0))
        res -= bd.Cylinder(r, self.top_thickness).translate((-hole_translate, holes_y_pos, 0))
 
        tolerance_term = 0.1
        w = self.plate_wedge(tolerance_term=tolerance_term)

        wedge_z_translate = -self.top_thickness/2 + self.wedge_height/2

        wedge_trans = self.top_width/2  - self.top_thickness
        res -= w.translate((-wedge_trans , 0, wedge_z_translate))
        res -= w.translate((wedge_trans,0, wedge_z_translate)) 

        return res
    

    def motor_box(self):
        res = bd.Part()
        t = self.top_thickness/2
        h = self.motor_height
        res += bd.Box(self.motor_width+t, self.motor_length+t, h)
        res -= bd.Box(self.motor_width, self.motor_length, h)
        res -= bd.Box(self.motor_width/2, t, h).translate((0,self.motor_length/2,h/4))

        return res.translate((0,0,h/2))


    def bottom_plate(self):
        res = bd.Part() + self.top_plate().rotate(bd.Axis.Y, 180)
        y_shift = self.holes_y_pos() - self.motor_length/2 - self.worm_gear_arm_length - self.worm_height/2
        res += self.motor_box().translate((0, y_shift , 0))
        return res
    
    def side(self):
        side = bd.Part() + bd.Box(self.top_thickness, self.top_depth * 0.80, self.gripper_height)

        side += (
            self.plate_wedge(length=self.top_depth*0.80)
            .translate((0,0,self.gripper_height/2+self.wedge_height/2))
        )

        side += (
            self.plate_wedge(length=self.top_depth*0.80)
            .rotate(bd.Axis.Y, 180)
            .translate((0,0,-self.gripper_height/2-self.wedge_height/2))
        )

        return side
    
    def export_parts(self, folder = "."):
        path = pathlib.Path(folder)
        
        gripper_left = self.make_gripper_left()
        gripper_right = self.make_gripper_right()
        gripper_top = self.top_plate().rotate(bd.Axis.Y, 180)
        gripper_bottom = self.bottom_plate()
        gripper_side = self.side().rotate(bd.Axis.Y, 90)
        worm_drive = self.worm_drive().rotate(bd.Axis.Y, 180)

        bd.export_stl(gripper_left, path / "gripper_left.stl")
        bd.export_stl(gripper_right, path / "gripper_right.stl")
        bd.export_stl(gripper_top, path / "gripper_top.stl")
        bd.export_stl(gripper_bottom, path / "gripper_bottom.stl")
        bd.export_stl(gripper_side, path / "gripper_side.stl")
        bd.export_stl(worm_drive, path / "worm_drive.stl")


grp = Gripper()

In [89]:
display(grp.side())

In [90]:
display(grp.bottom_plate())

In [91]:
display(grp.top_plate())

Compound at 0x1cb6fbfdfd0, label(), #children(0)

In [92]:

display(grp.make_gripper_left())

In [88]:
display(grp.make_gripper_right())

In [87]:
display(grp.worm_drive())

In [98]:
grp.export_parts("parts")